# Exploración Inicial y Limpieza de Datos

Notebook destinada al análisis exploratorio y proceso de limpieza de los datasets de Peliplat. Este proceso sigue metodologías ETL estándar para garantizar la calidad de los datos antes de su análisis posterior.

**Objetivos:**
- Explorar la estructura y calidad de los datasets de Peliplat
- Identificar y corregir problemas de codificación, duplicados y outliers
- Normalizar formatos de fecha y referencias entre datasets
- Preparar datos limpios para análisis posteriores

## 1. Setup inicial

En esta sección se importan las librerías necesarias y se cargan los datasets crudos. Se utiliza un enfoque modular importando funciones específicas del módulo `cleaning.py` para mantener el código organizado y reutilizable.

Los datasets cargados son:
- `users`: Información demográfica de usuarios
- `activity`: Métricas de actividad y comportamiento
- `campaigns`: Datos de campañas de marketing
- `content`: Catálogo de contenido disponible
- `engagement`: Interacciones de usuarios con el contenido

In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('..')
from src.data.cleaning import fix_encoding, cast_dates, remove_duplicates, process_outliers, save_processed_data, fix_temporal_inconsistencies


# Cargar datasets
users = pd.read_csv('../data/raw/peliplat_users.csv', encoding='latin1')
activity = pd.read_csv('../data/raw/peliplat_activity.csv', encoding='latin1')
campaigns = pd.read_csv('../data/raw/peliplat_campaigns.csv', encoding='latin1')
content = pd.read_csv('../data/raw/peliplat_content.csv', encoding='latin1')
engagement = pd.read_csv('../data/raw/peliplat_engagement.csv', encoding='latin1')


## 2. Exploración General Inicial

Esta sección realiza un análisis preliminar de los datasets para conocer su estructura y características principales. Para cada dataset se examina:

- Vista previa de las primeras filas (head)
- Estructura y tipos de datos (info)
- Detección de valores nulos


In [2]:
# Revisión rápida
datasets = {'users': users, 'activity': activity, 'campaigns': campaigns, 'content': content, 'engagement': engagement}

for name, df in datasets.items():
    print(f"Dataset {name}:")
    display(df.head())
    print(df.info())
    print(df.isna().sum())
    print('-'*50)


Dataset users:


,user_id,country,signup_date,last_active,gender,age
0,U0000,Chile,2025-03-11,2025-03-17,Male,51
1,U0001,Perú,2025-01-11,2025-03-30,Female,55
2,U0002,Colombia,2025-01-16,2025-03-14,Male,51
3,U0003,Perú,2025-03-14,2025-03-30,Female,35
4,U0004,Perú,2025-02-28,2025-03-20,Other,47


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      200 non-null    object
 1   country      200 non-null    object
 2   signup_date  200 non-null    object
 3   last_active  200 non-null    object
 4   gender       200 non-null    object
 5   age          200 non-null    int64 
dtypes: int64(1), object(5)
memory usage: 9.5+ KB
None
user_id        0
country        0
signup_date    0
last_active    0
gender         0
age            0
dtype: int64
--------------------------------------------------
Dataset activity:


,user_id,avg_daily_visits,avg_time_on_page,most_viewed_type
0,U0000,0.83,261.7,Lista
1,U0001,0.57,261.7,ArtÃ­culo
2,U0002,2.68,261.7,Video
3,U0003,0.97,261.7,Lista
4,U0004,0.83,261.7,Lista


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_id           200 non-null    object 
 1   avg_daily_visits  200 non-null    float64
 2   avg_time_on_page  200 non-null    float64
 3   most_viewed_type  200 non-null    object 
dtypes: float64(2), object(2)
memory usage: 6.4+ KB
None
user_id             0
avg_daily_visits    0
avg_time_on_page    0
most_viewed_type    0
dtype: int64
--------------------------------------------------
Dataset campaigns:


...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   campaign_id  5 non-null      object 
 1   name         5 non-null      object 
 2   start_date   5 non-null      object 
 3   end_date     5 non-null      object 
 4   clicks       5 non-null      int64  
 5   signups      5 non-null      int64  
 6   CTR          5 non-null      float64
dtypes: float64(1), int64(2), object(4)
memory usage: 412.0+ bytes
None
campaign_id    0
name           0
start_date     0
end_date       0
clicks         0
signups        0
CTR            0
dtype: int64
--------------------------------------------------
Dataset content:


,content_id,creator_id,type,category,publish_date,visits
0,C0000,CR21,ArtÃ­culo,Mainstream,2025-01-10,48
1,C0001,CR4,ArtÃ­culo,Documental,2025-01-30,55
2,C0002,CR1,Lista,Documental,2025-01-25,61
3,C0003,CR24,Lista,ClÃ¡sicos,2025-03-25,55
4,C0004,CR9,Lista,ClÃ¡sicos,2025-01-05,39


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   content_id    300 non-null    object
 1   creator_id    300 non-null    object
 2   type          300 non-null    object
 3   category      300 non-null    object
 4   publish_date  300 non-null    object
 5   visits        300 non-null    int64 
dtypes: int64(1), object(5)
memory usage: 14.2+ KB
None
content_id      0
creator_id      0
type            0
category        0
publish_date    0
visits          0
dtype: int64
--------------------------------------------------
Dataset engagement:


,user_id,content_id,likes,shares,comments
0,U0073,C0048,2,1,0
1,U0191,C0189,3,1,1
2,U0004,C0135,3,0,1
3,U0099,C0042,3,1,2
4,U0069,C0089,2,0,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     500 non-null    object
 1   content_id  500 non-null    object
 2   likes       500 non-null    int64 
 3   shares      500 non-null    int64 
 4   comments    500 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 19.7+ KB
None
user_id       0
content_id    0
likes         0
shares        0
comments      0
dtype: int64
--------------------------------------------------


## 3. Limpieza 

Esta sección implementa un proceso sistemático de limpieza de datos, aplicando técnicas estándar de la industria para garantizar la integridad y consistencia de los datasets. Se utilizan funciones modularizadas del módulo `cleaning.py` para mantener el código organizado y reproducible.

El proceso de limpieza sigue una secuencia lógica que aborda diferentes tipos de problemas de calidad de datos, comenzando por los más fundamentales (codificación) hasta los más específicos (outliers).

### 3.1 Limpieza de codificación

Esta etapa corrige problemas de codificación de caracteres en campos de texto, principalmente caracteres especiales del español (tildes, eñes) que aparecen mal codificados. Se utiliza la función `fix_encoding` que implementa la biblioteca `ftfy` (Fixes Text For You) para detectar y corregir automáticamente estos problemas.

La biblioteca `ftfy` es una herramienta especializada en reparación de texto que:
- Detecta y corrige problemas comunes de codificación UTF-8
- Resuelve problemas de texto que ha sido codificado múltiples veces
- Normaliza caracteres compuestos a su forma canónica

Problemas principales detectados y corregidos:
- "ArtÃ­culo" → "Artículo" 
- "CampaÃ±a" → "Campaña"
- "ClÃ¡sicos" → "Clásicos"

In [3]:
# Guardar una copia de los valores únicos ANTES de aplicar correcciones
print("=== VALORES ÚNICOS ANTES DE LA CORRECCIÓN ===")

print("\n--- USUARIOS (ORIGINAL) ---")
print("Country únicos:")
print(users['country'].unique())
print("\nGender únicos:")
print(users['gender'].unique())

print("\n--- ACTIVIDAD (ORIGINAL) ---")
print("Most viewed type únicos:")
print(activity['most_viewed_type'].unique())

print("\n--- CAMPAÑAS (ORIGINAL) ---")
print("Nombre de campañas únicos:")
print(campaigns['name'].unique())

print("\n--- CONTENIDO (ORIGINAL) ---")
print("Tipos de contenido únicos:")
print(content['type'].unique())
print("\nCategorías únicas:")
print(content['category'].unique())

# Aplicar la función fix_encoding a todos los dataframes
users_clean = fix_encoding(users)
activity_clean = fix_encoding(activity)
campaigns_clean = fix_encoding(campaigns)
content_clean = fix_encoding(content)
engagement_clean = fix_encoding(engagement)

print("\n\n=== VALORES ÚNICOS DESPUÉS DE LA CORRECCIÓN ===")

print("\n--- USUARIOS (CORREGIDOS) ---")
print("Country únicos (corregidos):")
print(users_clean['country'].unique())
print("\nGender únicos (corregidos):")
print(users_clean['gender'].unique())

print("\n--- ACTIVIDAD (CORREGIDA) ---")
print("Most viewed type únicos (corregidos):")
print(activity_clean['most_viewed_type'].unique())

print("\n--- CAMPAÑAS (CORREGIDAS) ---")
print("Nombre de campañas únicos (corregidos):")
print(campaigns_clean['name'].unique())

print("\n--- CONTENIDO (CORREGIDO) ---")
print("Tipos de contenido únicos (corregidos):")
print(content_clean['type'].unique())
print("\nCategorías únicas (corregidas):")
print(content_clean['category'].unique())

=== VALORES ÚNICOS ANTES DE LA CORRECCIÓN ===

--- USUARIOS (ORIGINAL) ---
Country únicos:
['Chile' 'Perú' 'Colombia' 'México' 'Argentina']

Gender únicos:
['Male' 'Female' 'Other']

--- ACTIVIDAD (ORIGINAL) ---
Most viewed type únicos:
['Lista' 'ArtÃ\xadculo' 'Video']

--- CAMPAÑAS (ORIGINAL) ---
Nombre de campañas únicos:
['CampaÃ±a_1' 'CampaÃ±a_2' 'CampaÃ±a_3' 'CampaÃ±a_4' 'CampaÃ±a_5']

--- CONTENIDO (ORIGINAL) ---
Tipos de contenido únicos:
['ArtÃ\xadculo' 'Lista' 'Video']

Categorías únicas:
['Mainstream' 'Documental' 'ClÃ¡sicos' 'Indie']


=== VALORES ÚNICOS DESPUÉS DE LA CORRECCIÓN ===

--- USUARIOS (CORREGIDOS) ---
Country únicos (corregidos):
['Chile' 'Perú' 'Colombia' 'México' 'Argentina']

Gender únicos (corregidos):
['Male' 'Female' 'Other']

--- ACTIVIDAD (CORREGIDA) ---
Most viewed type únicos (corregidos):
['Lista' 'Artículo' 'Video']

--- CAMPAÑAS (CORREGIDAS) ---
Nombre de campañas únicos (corregidos):
['Campaña_1' 'Campaña_2' 'Campaña_3' 'Campaña_4' 'Campaña_5']

---

### 3.2 Convertir fechas usando la función cast_dates

En esta etapa se convierten las columnas de tipo texto que contienen fechas a objetos datetime de pandas (formato datetime64[ns]). Este proceso es crucial ya que:

- Permite realizar operaciones temporales (diferencias, rangos, filtros)
- Facilita el análisis de tendencias y patrones temporales
- Habilita la detección de inconsistencias en datos cronológicos

Se procesan las siguientes columnas de fecha en cada dataset:
- **users**: 'signup_date', 'last_active'
- **campaigns**: 'start_date', 'end_date'
- **content**: 'publish_date'

La conversión se realiza de manera uniforme mediante la función `cast_dates`, que utiliza internamente `pd.to_datetime()` con gestión de errores para garantizar la integridad de los datos. Aunque las fechas ya se encontraban en formato ISO (YYYY-MM-DD), esta conversión es necesaria para habilitar operaciones de análisis temporal.### 3.2 Convertir fechas usando la función cast_dates


In [4]:
# Columnas con fechas en cada dataframe
users_date_cols = ['signup_date', 'last_active']
campaigns_date_cols = ['start_date', 'end_date']
content_date_cols = ['publish_date']

# Aplicar la función cast_dates a los dataframes limpios
users_clean = cast_dates(users_clean, users_date_cols)
campaigns_clean = cast_dates(campaigns_clean, campaigns_date_cols)
content_clean = cast_dates(content_clean, content_date_cols)

# Verificar que las fechas se han convertido correctamente
print("=== VERIFICACIÓN DE CONVERSIÓN DE FECHAS ===")

print("\n--- USUARIOS ---")
print("Antes de la conversión:")
print(users['signup_date'].head())
print(users['signup_date'].dtype)
print("\nDespués de la conversión:")
print(users_clean['signup_date'].head())
print(users_clean['signup_date'].dtype)

print("\n--- CAMPAÑAS ---")
print("Antes de la conversión:")
print(campaigns['start_date'].head())
print(campaigns['start_date'].dtype)
print("\nDespués de la conversión:")
print(campaigns_clean['start_date'].head())
print(campaigns_clean['start_date'].dtype)

print("\n--- CONTENIDO ---")
print("Antes de la conversión:")
print(content['publish_date'].head())
print(content['publish_date'].dtype)
print("\nDespués de la conversión:")
print(content_clean['publish_date'].head())
print(content_clean['publish_date'].dtype)

=== VERIFICACIÓN DE CONVERSIÓN DE FECHAS ===

--- USUARIOS ---
Antes de la conversión:
0   2025-03-11
1   2025-01-11
2   2025-01-16
3   2025-03-14
4   2025-02-28
Name: signup_date, dtype: datetime64[ns]
datetime64[ns]

Después de la conversión:
0   2025-03-11
1   2025-01-11
2   2025-01-16
3   2025-03-14
4   2025-02-28
Name: signup_date, dtype: datetime64[ns]
datetime64[ns]

--- CAMPAÑAS ---
Antes de la conversión:
0   2025-01-10
1   2025-01-25
2   2025-02-10
3   2025-02-20
4   2025-03-10
Name: start_date, dtype: datetime64[ns]
datetime64[ns]

Después de la conversión:
0   2025-01-10
1   2025-01-25
2   2025-02-10
3   2025-02-20
4   2025-03-10
Name: start_date, dtype: datetime64[ns]
datetime64[ns]

--- CONTENIDO ---
Antes de la conversión:
0   2025-01-10
1   2025-01-30
2   2025-01-25
3   2025-03-25
4   2025-01-05
Name: publish_date, dtype: datetime64[ns]
datetime64[ns]

Después de la conversión:
0   2025-01-10
1   2025-01-30
2   2025-01-25
3   2025-03-25
4   2025-01-05
Name: publish_date

### 3.3 Duplicados

Esta etapa identifica y elimina registros duplicados en los datasets, enfocándose particularmente en el dataset de engagement donde se encontraron duplicados.

El proceso utiliza la función `remove_duplicates` que:
- Identifica duplicados basados en columnas clave específicas para cada dataset
- Proporciona información detallada sobre grupos de duplicados encontrados
- Elimina duplicados manteniendo el primer registro por defecto (`keep='first'`)

En el análisis realizado:
- Se detectaron 2 duplicados en el dataset de **engagement** (2 grupos donde el mismo usuario interactuó con el mismo contenido)
- Estos duplicados fueron eliminados, preservando solo el primer registro de cada grupo
- Los demás datasets no presentaron registros duplicados en sus claves primarias

La eliminación de duplicados garantiza la integridad de los datos para análisis posteriores, evitando sesgos en métricas de comportamiento de usuario y relaciones entre entidades.

Nota: Aunque la función `remove_duplicates` tiene capacidad para consolidar duplicados sumando valores numéricos, en este análisis se optó por la eliminación simple.

In [5]:
# Verificar duplicados en los dataframes y eliminarlos si es necesario

# Verificar duplicados por user_id en users
print("=== VERIFICACIÓN DE DUPLICADOS ===")
print("\n--- USUARIOS ---")
print(f"Número de duplicados en user_id: {users_clean.duplicated(subset=['user_id']).sum()}")

# Verificar duplicados por user_id en activity
print("\n--- ACTIVIDAD ---")
print(f"Número de duplicados en user_id: {activity_clean.duplicated(subset=['user_id']).sum()}")

# Verificar duplicados por campaign_id en campaigns
print("\n--- CAMPAÑAS ---")
print(f"Número de duplicados en campaign_id: {campaigns_clean.duplicated(subset=['campaign_id']).sum()}")

# Verificar duplicados por content_id en content
print("\n--- CONTENIDO ---")
print(f"Número de duplicados en content_id: {content_clean.duplicated(subset=['content_id']).sum()}")

# Verificar duplicados en engagement (combinación de user_id y content_id)
print("\n--- ENGAGEMENT ---")
print(f"Número de duplicados en user_id+content_id: {engagement_clean.duplicated(subset=['user_id', 'content_id']).sum()}")

# Eliminar duplicados si es necesario
if users_clean.duplicated(subset=['user_id']).sum() > 0:
    users_clean = remove_duplicates(users_clean, subset=['user_id'])
    print("Duplicados eliminados en users")

if activity_clean.duplicated(subset=['user_id']).sum() > 0:
    activity_clean = remove_duplicates(activity_clean, subset=['user_id'])
    print("Duplicados eliminados en activity")

if campaigns_clean.duplicated(subset=['campaign_id']).sum() > 0:
    campaigns_clean = remove_duplicates(campaigns_clean, subset=['campaign_id'])
    print("Duplicados eliminados en campaigns")

if content_clean.duplicated(subset=['content_id']).sum() > 0:
    content_clean = remove_duplicates(content_clean, subset=['content_id'])
    print("Duplicados eliminados en content")

if engagement_clean.duplicated(subset=['user_id', 'content_id']).sum() > 0:
    engagement_clean = remove_duplicates(engagement_clean, subset=['user_id', 'content_id'])
    print("Duplicados eliminados en engagement")

# Verificar la cantidad de registros después de eliminar duplicados
print("\n=== RECUENTO DE REGISTROS DESPUÉS DE ELIMINAR DUPLICADOS ===")
print(f"users_clean: {len(users_clean)} registros")
print(f"activity_clean: {len(activity_clean)} registros")
print(f"campaigns_clean: {len(campaigns_clean)} registros")
print(f"content_clean: {len(content_clean)} registros")
print(f"engagement_clean: {len(engagement_clean)} registros")

=== VERIFICACIÓN DE DUPLICADOS ===

--- USUARIOS ---
Número de duplicados en user_id: 0

--- ACTIVIDAD ---
Número de duplicados en user_id: 0

--- CAMPAÑAS ---
Número de duplicados en campaign_id: 0

--- CONTENIDO ---
Número de duplicados en content_id: 0

--- ENGAGEMENT ---
Número de duplicados en user_id+content_id: 2
🔍 Se encontraron 2 registros duplicados en 2 grupos.

⚠️ Duplicados encontrados:
    user_id content_id  likes  shares  comments
74    U0020      C0257      1       0         1
450   U0020      C0257      5       1         2
--------------------------------------------------

⚠️ Duplicados encontrados:
    user_id content_id  likes  shares  comments
81    U0146      C0126      3       1         1
350   U0146      C0126      4       0         2
--------------------------------------------------
Duplicados eliminados en engagement

=== RECUENTO DE REGISTROS DESPUÉS DE ELIMINAR DUPLICADOS ===
users_clean: 200 registros
activity_clean: 200 registros
campaigns_clean: 5 regis

### 3.4 Corrección de Inconsistencias Temporales

Esta etapa identifica y corrige inconsistencias lógicas en los datos temporales, específicamente usuarios cuya fecha de última actividad es anterior a su fecha de registro. 

La metodología implementada utiliza la función `fix_temporal_inconsistencies` que:
- Calcula la diferencia en días entre la última actividad y el registro
- Identifica registros con diferencias negativas (inconsistencias lógicas)
- Ajusta la fecha de registro para que coincida con la fecha de última actividad

Este enfoque de corrección se basa en las siguientes premisas:
1. Los datos de actividad suelen ser más precisos que los de registro
2. Se preservan todos los registros evitando sesgos en métricas 
3. Se siguen recomendaciones de estudios recientes sobre análisis de engagement

Se identificaron y corrigieron 6 registros con inconsistencias temporales, donde la fecha de última actividad era anterior a la fecha de registro. En vez de eliminar estos registros, se ajustó la fecha de registro al valor de la última actividad, asumiendo que esta representa la primera interacción real del usuario con la plataforma.

Esta corrección metodológica es fundamental para análisis de retención y comportamiento a lo largo del tiempo, garantizando que las métricas de permanencia y actividad sean lógicamente consistentes.

In [6]:
# 1. Calcular días activos y mostrar inconsistencias antes de corregir
users_clean['days_active'] = (users_clean['last_active'] - users_clean['signup_date']).dt.days
usuarios_inconsistentes = users_clean[users_clean['days_active'] < 0].copy()

# Mostrar registros problemáticos
print("Registros con fechas inconsistentes (última actividad anterior al registro):")
print(usuarios_inconsistentes[['user_id', 'country', 'signup_date', 'last_active', 'gender', 'age', 'days_active']])
print(f"\nTotal de registros inconsistentes: {len(usuarios_inconsistentes)}")

# 2. Aplicar corrección metodológica usando la función modularizada
users_clean = fix_temporal_inconsistencies(users_clean, 'signup_date', 'last_active', verbose=True)

# 3. Verificar corrección
print(f"Registros con días activos negativos después de corrección: {len(users_clean[users_clean['days_active'] < 0])}")

Registros con fechas inconsistentes (última actividad anterior al registro):
    user_id    country signup_date last_active  gender  age  days_active
5     U0005     México  2025-03-11  2025-03-05  Female   32           -6
14    U0014      Chile  2025-03-12  2025-03-01  Female   34          -11
53    U0053       Perú  2025-03-02  2025-03-01  Female   38           -1
113   U0113   Colombia  2025-03-07  2025-03-04  Female   51           -3
161   U0161  Argentina  2025-03-09  2025-03-05  Female   43           -4
198   U0198       Perú  2025-03-10  2025-03-07   Other   47           -3

Total de registros inconsistentes: 6
🔍 Se encontraron 6 registros con inconsistencias temporales.

⚠️ Ejemplos de inconsistencias:
    user_id signup_date last_active  days_active
5     U0005  2025-03-11  2025-03-05           -6
14    U0014  2025-03-12  2025-03-01          -11
53    U0053  2025-03-02  2025-03-01           -1
113   U0113  2025-03-07  2025-03-04           -3
161   U0161  2025-03-09  2025-03-05

### 3.5 Tratamiento de Outliers

Esta etapa identifica y trata valores atípicos (outliers) en las variables numéricas de los datasets. Se implementa un enfoque de winsorización selectiva mediante la función `process_outliers` que:

- Aplica factores específicos por columna para definir umbrales de detección
- Utiliza el método de Rango Intercuartílico (IQR) multiplicado por factores configurables
- Reemplaza valores extremos por los límites definidos en lugar de eliminarlos
- Preserva la distribución general de los datos sin perder observaciones

La configuración aplicada utiliza diferentes factores según el tipo de variable:
- Métricas de actividad (`avg_daily_visits`): factor IQR de 3.0
- Visitas a contenido (`visits`): factor IQR de 3.0
- Métricas de engagement (`likes`, `shares`, `comments`): factor IQR de 4.0

Esta estrategia diferenciada refleja la naturaleza específica de cada variable:
- Mayor tolerancia (factor 4.0) para métricas de engagement donde la variabilidad es esperada
- Menor tolerancia (factor 3.0) para métricas de actividad y visitas

El análisis no identificó outliers según estos criterios, lo que indica que los datos ya presentaban distribuciones razonablemente consistentes dentro de los umbrales establecidos.

Esta etapa es crucial para garantizar que análisis estadísticos posteriores no se vean distorsionados por valores extremos, manteniendo la robustez de los resultados analíticos.

In [7]:
# Corregir la estructura de los DataFrames si son tuplas
dfs = {'users': users_clean, 'activity': activity_clean, 'campaigns': campaigns_clean, 
       'content': content_clean, 'engagement': engagement_clean}

# Desempaquetar cualquier tupla (resultados de remove_duplicates)
for key in dfs:
    if isinstance(dfs[key], tuple):
        dfs[key], _ = dfs[key]

# Definir factores específicos para cada columna
column_factors = {
    'activity': {'avg_daily_visits': 3.0},
    'content': {'visits': 3.0},
    'engagement': {'likes': 4.0, 'shares': 4.0, 'comments': 4.0}
}

# Procesar outliers en todos los DataFrames y obtener resultados finales
clean_dfs = process_outliers(dfs, column_factors)

# Asignar los DataFrames limpios a variables individuales
users_clean_final = clean_dfs['users']
activity_clean_final = clean_dfs['activity']
campaigns_clean_final = clean_dfs['campaigns']
content_clean_final = clean_dfs['content']
engagement_clean_final = clean_dfs['engagement']

=== 🔍 TRATAMIENTO DE OUTLIERS (MEJORES PRÁCTICAS) ===

--- 📋 USERS ---

--- 📋 ACTIVITY ---

--- 📊 ACTIVITY (AVG_DAILY_VISITS) ---

📊 Estadísticas de avg_daily_visits antes de winsorización:
count    200.000000
mean       1.551700
std        0.852482
min        0.200000
25%        0.810000
50%        1.480000
75%        2.275000
max        2.990000
Name: avg_daily_visits, dtype: float64
🔍 Límites para outliers (IQR x 3.0): -3.58 - 6.67
🔢 Número de outliers detectados: 0
✅ No se detectaron outliers.

--- 📋 CAMPAIGNS ---

--- 📋 CONTENT ---

--- 📊 CONTENT (VISITS) ---

📊 Estadísticas de visits antes de winsorización:
count    300.000000
mean      50.016667
std        6.954442
min       33.000000
25%       45.000000
50%       50.000000
75%       55.000000
max       74.000000
Name: visits, dtype: float64
🔍 Límites para outliers (IQR x 3.0): 15.00 - 85.00
🔢 Número de outliers detectados: 0
✅ No se detectaron outliers.

--- 📋 ENGAGEMENT ---

--- 📊 ENGAGEMENT (LIKES) ---

📊 Estadísticas de like

## 4. Almacenamiento de Datos Procesados

En esta etapa final se guardan los datasets limpios y procesados para su uso en análisis posteriores. Se utiliza la función `save_processed_data` que:

- Almacena cada dataset en formato CSV con codificación UTF-8
- Aplica una nomenclatura estandarizada (prefijo "peliplat_" y sufijo "_clean")
- Genera la estructura de directorios necesaria si no existe
- Proporciona confirmación detallada del proceso de guardado

Los datasets procesados y guardados son:
- `peliplat_users_clean.csv`: 200 registros
- `peliplat_activity_clean.csv`: 200 registros
- `peliplat_campaigns_clean.csv`: 5 registros
- `peliplat_content_clean.csv`: 300 registros
- `peliplat_engagement_clean.csv`: 498 registros

Estos archivos constituyen la base para el análisis exploratorio avanzado y modelado, cumpliendo con los requisitos de calidad de datos establecidos:
- Codificación correcta de caracteres especiales
- Fechas en formato datetime estandarizado
- Eliminación de duplicados innecesarios
- Corrección de inconsistencias temporales
- Tratamiento adecuado de valores extremos

El almacenamiento en la carpeta `data/processed/` sigue las convenciones de organización de proyectos de análisis de datos, separando claramente datos crudos de datos procesados.

In [8]:
# Crear un diccionario con los DataFrames limpios finales
dfs_limpios = {
    'users': users_clean_final,
    'activity': activity_clean_final, 
    'campaigns': campaigns_clean_final,
    'content': content_clean_final,
    'engagement': engagement_clean_final
}

# Guardar todos los DataFrames en la carpeta data/processed/
archivos_guardados = save_processed_data(dfs_limpios)

✅ Guardado exitoso: peliplat_users_clean.csv (200 registros)
✅ Guardado exitoso: peliplat_activity_clean.csv (200 registros)
✅ Guardado exitoso: peliplat_campaigns_clean.csv (5 registros)
✅ Guardado exitoso: peliplat_content_clean.csv (300 registros)
✅ Guardado exitoso: peliplat_engagement_clean.csv (498 registros)

📂 Archivos guardados en: C:\Users\trico\Desktop\Prueba cine proyecto\data\processed
